In [ ]:
import sys

sys.path.append('..')
from my_util import my_read, my_write, MultiProcessing
from my_occurance import Occurance, occurance_dump, occurance_load, occurance_post_operation

In [ ]:
# Generate sentence file with line number
!grep -n '' ../data/corpus/small_sent.txt > ../data/corpus/small_sent_line.txt

In [ ]:
# Generate occurance file
# To run the code in the backend, use the gen_occur.py in the "py" folder
p = MultiProcessing()
occur_dict = p.run(lambda: Occurance('../data/corpus/wordtree.json', '../data/corpus/keyword_f.txt'), open('../data/corpus/small_sent_line.txt').readlines(), 8, occurance_post_operation)
occurance_dump('../data/corpus/occur.json', occur_dict)

In [ ]:
# Remove the indexed file to avoid confusion
!rm ../data/corpus/small_sent_line.txt

In [ ]:
# Load sentence file and occurance file
sent_list = my_read('../data/corpus/small_sent.txt')
occur_dict = {line.split(':')[0] : set(map(int, line.split(':')[1].split())) for line in open('../data/corpus/occur.txt', 'r').read().split('\n')}

Play around in the below

In [ ]:
test_data = my_read('../data/test/co_occur_test.txt')
test_data = [data.split(',') for data in test_data]
test_dict = {data[0] : data[1:] for data in test_data}

In [ ]:
test_sent_dict = {central_kw : set() for central_kw in test_dict}
for central_kw, kws in test_dict.items():
    for kw in kws:
        test_sent_dict[central_kw] |= occur_dict[kw]
    test_sent_dict[central_kw] &= occur_dict[central_kw]

for central_kw, sents in test_sent_dict.items():
    content = [sent_list[i] for i in sents]
    my_write('../data/temp/%s_wiki.txt' % central_kw.replace(' ', '_'), content)

In [ ]:
test_lines = occur_dict['python'] & (occur_dict['java'] | occur_dict['ruby'])
my_write('python_java_ruby.txt', [sent_list[i] for i in test_lines])

Analyze the sentences with OLLIE, Stanford OpenIE or OpenIE5

In [ ]:
openie_data = my_read('../data/temp/pl_wiki_ollie_triple.txt')
# openie_data = my_read('pjr_ollie_triple.txt')
# keywords = set(['data structure', 'binary tree', 'hash table', 'linked list'])
keywords = set(['programming language', 'python', 'java', 'javascript', 'lua', 'scala', 'lisp', 'php', 'ruby', 'smalltalk'])
# keywords = set(['python', 'java', 'ruby'])

qualified_triples = []
for data in openie_data:
    if data:
        arg1, rel, arg2 = data.split(';')
        for kw in keywords:
            if kw in arg1:
                for kw in keywords:
                    if kw in arg2:
                        qualified_triples.append(data)
                        break
                break
my_write('pl_wiki_ollie_triple_f.txt', qualified_triples)

In [ ]:
data_structure_idx = occur_dict['data structure']

In [ ]:
co_occur_set = {}
for keyword, idx_set in occur_dict.items():
    intersection = idx_set & data_structure_idx
    if intersection:
        co_occur_set[keyword] = list(intersection)

In [ ]:
len(co_occur_set)

In [ ]:
sorted_co_occur_list = sorted(co_occur_set.items(), key=lambda x: len(x[1]), reverse=True)[:100]
sorted_co_occur_count = [(word, len(idx)) for word, idx in sorted_co_occur_list]

In [ ]:
sorted_co_occur_count[:40]

In [ ]:

'b-tree' in co_occur_set

In [ ]:
# sent_list[co_occur_set['b-tree'][0]]
temp_list = [sent_list[idx] for idx in co_occur_set['b-tree']]
my_write('ds_bt_sent.txt', temp_list)